In [5]:
import os
import zipfile

In [7]:
import os
import tarfile

# Replace 'path_to_input_folder' with the actual path to the folder containing the .tar files
input_folder = r"C:\Users\NimeshaJ\Code\L8"
# Replace 'path_to_output_folder' with the desired path where the extracted data should be saved
output_folder = r"C:\Users\NimeshaJ\Code\L8p"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Get a list of all files in the input folder
file_list = os.listdir(input_folder)

# Loop through each file in the input folder
for file_name in file_list:
    # Check if the file is a .tar file
    if file_name.endswith('.tar'):
        tar_file_path = os.path.join(input_folder, file_name)
        
        # Create a subfolder with the same name as the .tar file (without the .tar extension)
        subfolder_name = os.path.splitext(file_name)[0]
        output_subfolder = os.path.join(output_folder, subfolder_name)
        os.makedirs(output_subfolder, exist_ok=True)
        
        # Extract the contents of the .tar file to the subfolder
        with tarfile.open(tar_file_path, 'r') as tar_ref:
            tar_ref.extractall(output_subfolder)

In [12]:
!pip install geopandas



                                              0.0/1.1 MB ? eta -:--:--
                                              0.0/1.1 MB ? eta -:--:--
     -                                        0.0/1.1 MB 330.3 kB/s eta 0:00:04
     --                                       0.1/1.1 MB 469.7 kB/s eta 0:00:03
     ----------------------------             0.8/1.1 MB 4.2 MB/s eta 0:00:01
     ---------------------------------------  1.1/1.1 MB 5.3 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 4.1 MB/s eta 0:00:00
                                              0.0/22.7 MB ? eta -:--:--
     --                                       1.4/22.7 MB 30.9 MB/s eta 0:00:01
     ----                                     2.5/22.7 MB 31.6 MB/s eta 0:00:01
     ----                                     2.5/22.7 MB 31.6 MB/s eta 0:00:01
     -----                                    3.0/22.7 MB 15.9 MB/s eta 0:00:02
     --------                                 4.8/22.7 MB 20.3 MB/s eta 

In [13]:
import os
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.mask import mask

In [14]:
shapefile = gpd.read_file(r"C:\Ibrahim\HongKongB.shp")

In [17]:
shapefile.geometry

0    POLYGON ((791646.304 2453369.232, 791646.304 2...
Name: geometry, dtype: geometry

In [18]:
def process_landsat_folder(folder_path, shapefile_path, output_folder):
    # Read the Landsat bands and perform cropping and LST estimation for a single folder
    # Replace the LST estimation part with your Split Window Algorithm implementatio

    # Get a list of all files in the Landsat folder
    file_list = os.listdir(folder_path)

    # Read the shapefile using geopandas
    shapefile = gpd.read_file(shapefile_path)

    # Loop through each file in the Landsat folder
    for file_name in file_list:
        # Check if the file is a GeoTIFF (band file)
        if file_name.endswith('.tif'):
            tif_file_path = os.path.join(folder_path, file_name)
            
            # Read the band file using rasterio
            with rasterio.open(tif_file_path) as src:
                # Get the band number from the file name
                band_number = int(file_name.split('_')[-1].split('.')[0])

                # Check if the band is 3, 4, 10, or 11
                if band_number in [3, 4, 10, 11]:
                    # Crop the band using the shapefile
                    cropped_band, _ = mask(src, shapefile.geometry, crop=True, indexes=band_number)

                    # Estimate LST (Replace this part with your Split Window Algorithm implementation)
                    # For demonstration, we assume LST estimation is done as the mean of pixel values
                    lst = np.mean(cropped_band)

                    # Create a new file name for the LST output
                    lst_output_file = file_name.replace(f"_B{band_number}.tif", "_LST.tif")
                    lst_output_path = os.path.join(output_folder, lst_output_file)

                    # Save the LST as a new GeoTIFF file
                    with rasterio.open(lst_output_path, 'w', **src.meta) as dst:
                        dst.write(lst, 1)

if __name__ == "__main__":
    # Replace 'path_to_directory' with the actual path to the directory containing Landsat 8 folders
    directory_path = 'path_to_directory'
    # Replace 'path_to_shapefile.geojson' with the path to your GeoJSON shapefile
    shapefile_path = 'path_to_shapefile.geojson'
    output_folder = r"C:\Users\NimeshaJ\Code\LST"

    # Get a list of all folders in the specified directory
    folder_list = os.listdir(directory_path)

    # Process each Landsat folder one by one
    for folder_name in folder_list:
        folder_path = os.path.join(directory_path, folder_name)
        process_landsat_folder(folder_path, shapefile_path)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'path_to_directory'

In [19]:
def ndvi(b4,b5):
    b4 = ((0.00002 * b4)- 0.1) / 0.786774820629
    b5 = ((0.00002 * b5) - 0.1) / 0.786774820629
    return float(b5-b4)/float(b5+b4)   

In [ ]:
def emm(ndvi,):
    pv = ((ndvi - 2)/(0.5 - 0.2))**2
    b10_emppp = (0.984 * pv) + 0.964 * (1 - pv) + (1 - 0.964) * (1 - pv) * 0.05 * 0.984
    
    #condition for estimating LSE band 10
    condition_1 = ndvi < 0
    result_1 = np.where(condition_1, 0.991, 0.0)  
    # Second condition: ("%ndviii%" >= 0) & ("%ndviii%" < 0.2) -> True where ndviii is between 0 and 0.2
    condition_2 = (ndvi >= 0) & (ndvi < 0.2)
    result_2 = np.where(condition_2, 0.964, 0.0)  # Replace np.nan with 0.0 for False values

    # Third condition: ("%ndviii%" >= 0.2) & ("%ndviii%" <= 0.5) -> True where ndviii is between 0.2 and 0.5
    condition_3 = (ndvi >= 0.2) & (ndvi <= 0.5)
    result_3 = np.where(condition_3, b10_emppp, 0.0)  # Replace np.nan with 0.0 for False values

    # Fourth condition: ("%ndviii%" > 0.5) -> True where ndviii is greater than 0.5
    condition_4 = ndvi > 0.5
    result_4 = np.where(condition_4, 0.984, 0.0)  # Replace np.nan with 0.0 for False values

    # Combine the results using np.nanmean, which calculates the mean while ignoring 0.0 values (False values)
    b10_em = np.nanmean([result_1, result_2, result_3, result_4], axis=0)
    b11_empp = (0.)